<a href="https://colab.research.google.com/github/nelangovan/word2vec/blob/master/Workbook_Word_Meaning_and_Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Meaning and Word2vec

This notebook provides a general introduction to natural language processing (NLP) using Word2vec. If you haven't finished the first module, [Deep Learning and Natural Language Processing](https://trailhead.salesforce.com/content/learn/modules/deep-learning-and-natural-language-processing), we highly suggest starting there.

Code sections of the notebook appear in grey cells. To run the code in a cell, hover over the brackets in the upper left corner of the cell and click the play button or **Shift+Enter**. You can edit the code in any cell. When running a cell, be sure that you've run all the above cells first to avoid errors.

When you have completed the lab, return to Trailhead to enter your answers to the exercises in the quiz section and get points.

In [0]:
import matplotlib.pyplot as plt
import random
import collections
import numpy as np
import os
import urllib
import zipfile
import collections
import math
import os
import datetime as dt
import string
import re
import time

import numpy as np



!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.module import Module

def println(*x):
  print(*x)
  print()

#  Extract data

### Data Description
We'll be playing around with two datasets to get a sense of how the underlying data affects the training of the word vectors. The first dataset is a large chunk of text that comes from English Wikipedia, and the second is a set of movie reviews from the Stanford Sentiment Treebank (SST).

### Download the data

In [0]:
en_wiki_url='http://mattmahoney.net/dc/text8.zip'
sst_url = 'https://raw.githubusercontent.com/salesforce/decaNLP/master/local_data/train_fine_sent.csv'

def download(url):
    filename = os.path.basename(url)
    if not os.path.exists(filename):
        downloaded_path, _ = urllib.request.urlretrieve(url, filename)
    else:
      downloaded_path = filename
    return downloaded_path

en_wiki_path = download(en_wiki_url)
sst_path = download(sst_url)

print('English Wikipedia (small): ', en_wiki_path)
print('Stanford Sentiment Treebank: ', sst_path)

English Wikipedia (small):  text8.zip
Stanford Sentiment Treebank:  train_fine_sent.csv


### Extract sentences

In [0]:
en_wiki_sentences, sst_sentences = None, None

# The English Wikipedia data has already been cleaned, tokenized, and turned 
# into a long string of words that can be considered a single sentence
with zipfile.ZipFile(en_wiki_path) as f:
  words = f.read(f.namelist()[0]).split()
  en_wiki_sentences = [[w.decode() for w in words]]

# We need to clean the data from the movie reviews
def clean(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))
  
  
with open(sst_path) as f:
  # skip the labels of the csv file
  next(f) 
  # skip the label and the comma, then clean
  sst_sentences = [clean(s[2:]).split() for s in f] 

In [0]:
print('English Wikipedia (small) example: ', en_wiki_sentences[0][:10])
print(len(en_wiki_sentences[0]))
print('Stanford Sentiment Treebank example: ', sst_sentences[0][:10])

English Wikipedia (small) example:  ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
17005207
Stanford Sentiment Treebank example:  ['stirring', 'funny', 'and', 'finally', 'transporting', 'reimagining', 'of', 'beauty', 'and', 'beast']


# Quiz Question 1:
What is the first word in the first example of the English Wikipedia dataset?

# Quiz Question 2:
What is the first word in the first example of the SST dataset?

# Create a vocabulary

Now that we've downloaded and extracted our data, we need to construct a vocabulary. This code creates two vocabularies for each dataset—one that contains all the words that appear in the sentences, and one that contains only words that appear five or more times. Then we'll take a look at both the most common words in each vocabulary, as well as what our earlier example sentences look like with uncommon words removed.

In [0]:
class Vocabulary:
  
  def __init__(self, sentences):
    word_counts = collections.Counter([w for s in sentences for w in s ])
    print('Sentences contain ', len(word_counts), ' words.')
    common_word_counts = {word: count 
                          for word, count in word_counts.items() if count >= 5}
    print('Sentences contain ', len(common_word_counts), ' words that occur at least 5 times.')
                                 
    # Replace uncommon words with a special unknown token
    unk_token = 'UNK'
    unk_token_count = 0
    new_sentences = []
    for sentence in sentences:
      new_sentence = []
      for word in sentence:
        if word in common_word_counts:
          new_sentence.append(word)
        else:
          unk_token_count += 1
          new_sentence.append(unk_token)
      new_sentences.append(new_sentence)
    self.sentences = new_sentences
                            
    if unk_token in common_word_counts:
      common_word_counts[unk_token] += unk_token_count
    else:
      common_word_counts[unk_token] = unk_token_count
    num_tokens = sum(common_word_counts.values())
                                     
    self.index_to_word, self.word_to_index, self.word_to_frequency = {}, {}, {}
    sorted_common_word_counts = sorted(common_word_counts.items(), 
                                       key=lambda tup: (-tup[1], tup[0]))

    for idx, (word, count) in enumerate(sorted_common_word_counts):
      self.index_to_word[idx] = word
      self.word_to_index[word] = idx 
      self.word_to_frequency[word] = count / num_tokens

print('Wikipedia (full | uncommon words removed)')
en_wiki_vocab = Vocabulary(en_wiki_sentences)
print('\nSST (full | uncommon words removed)')
sst_vocab = Vocabulary(sst_sentences)

en_wiki_unked = en_wiki_vocab.sentences
sst_unked = sst_vocab.sentences

Wikipedia (full | uncommon words removed)
Sentences contain  253854  words.
Sentences contain  71290  words that occur at least 5 times.

SST (full | uncommon words removed)
Sentences contain  17008  words.
Sentences contain  3385  words that occur at least 5 times.


In [0]:
print('Wikipedia Vocabulary')
print(len(en_wiki_vocab.index_to_word))
print(type(en_wiki_vocab.index_to_word))
print(list(en_wiki_vocab.index_to_word[i] for i in range(10)))
print(en_wiki_unked[0][:10])

print('\nSST Vocabulary')
print(len(sst_vocab.index_to_word))
print(list(sst_vocab.index_to_word[i] for i in range(10)))
print(sst_unked[0][:10])

Wikipedia Vocabulary
71291
<class 'dict'>
['the', 'of', 'and', 'one', 'in', 'a', 'to', 'UNK', 'zero', 'nine']
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']

SST Vocabulary
3386
['UNK', 'and', 'of', 'to', 'is', 'in', 'that', 'its', 'it', 'as']
['stirring', 'funny', 'and', 'finally', 'UNK', 'UNK', 'of', 'beauty', 'and', 'beast']


# Quiz Question 3:
What is the most common word in the English Wikipedia vocabulary constructed above? 

# Quiz Question 4:
Which words were replaced by the UNK token in the first SST sentence?

# Drop words based on frequency

The most frequent words usually provide less information. For
example, nearly every word co-occurs frequently with “the”. 
For this reason, we discard words with a probability related to its frequency and a threshold typically set to 10e-5.

In [0]:
def drop_words(sentences, vocabulary, threshold=10e-5):
  new_sentences = []
  for sentence in sentences:
    new_sentence = []
    for word in sentence:
      word_frequency = vocabulary.word_to_frequency[word]
      discard_probability = 1 - np.sqrt(threshold / word_frequency)
      if random.random() > discard_probability:
        new_sentence.append(word)
    if len(new_sentence) > 0:
        new_sentences.append(new_sentence)
  return new_sentences

random.seed(123)

en_wiki_dropped = drop_words(en_wiki_unked, en_wiki_vocab)
sst_dropped = drop_words(sst_unked, sst_vocab)

In [0]:
print('Wikipedia vocabulary with dropped words')
print(en_wiki_dropped[0][:10])

print('\nSST vocabulary with dropped words')
print(sst_dropped[0][:10])

Wikipedia vocabulary with dropped words
['anarchism', 'originated', 'term', 'abuse', 'used', 'working', 'radicals', 'diggers', 'revolution', 'sans']

SST vocabulary with dropped words
['stirring', 'finally', 'beauty', 'beast']


# Quiz Question 5:
What was the first word dropped from the English WIkipedia example?

# Quiz Question 6:
What was the first word dropped from the SST example?

# Hands-on: Numericalize the data

It's your turn to write some code! Implement the ```numericalize``` method. This method takes in a sentence and a vocabulary and returns a list containing the indices of each word in the sentence, in the same order, according to the vocabulary.

In [0]:
def numericalize(sentence, vocabulary):
  numericalized = []
  for s in sentence:
    if s in vocabulary.word_to_index.keys():
      numericalized.append(vocabulary.word_to_index[s])
  return numericalized

In [0]:
en_wiki_numericalized = [numericalize(s, en_wiki_vocab) for s in en_wiki_dropped]
sst_numericalized = [numericalize(s, sst_vocab) for s in sst_dropped]
#en_wiki_numericalized = np.array(en_wiki_numericalized[0])

In [0]:
print(en_wiki_numericalized[0][:10])
print(sst_numericalized[0][:10])
print(len(en_wiki_numericalized))
print(len(sst_numericalized))


[5234, 3084, 195, 3134, 59, 742, 10620, 27498, 855, 15178]
[1647, 513, 658, 2117]
1
8343


# Quiz Question 7:
What is the first index in the English Wikipedia example after numericalization?

# Word2Vec Variations
There are four variations of training Word2Vec.

Your Word2Vec model can either be skip-gram (nSG) or continuous bag-of-words (nCBOW) where n represents the wondow size.
It can also be trained with a full softmax (FS) or with k-negative sampling (kNS). The difference between these variants really comes down to how training examples are constructed.

In order to construct a training example, we:


1. Choose a random sentence from the dataset.
2. Choose a random word in the sentence  and consider the context window to be the word along with up to n-1 words to its right. If the context window has length less than or equal to n//2, return to Step 1.
3. Let center_word be the word at the middle of the context window.
4. Let context_words be the set of all other words in the context window.
5. Let context_word be one of the words in context_words chosen at random.
5. If using the nSG model, let example be [center_word, context_word].
6. If using the nCBOW model, let example be [context_words, center_word].
7. If using a kNS model, extend example to be a triple with the final entry a list of k randomly sampled words from the vocabulary.


* The nSG models predicts context words from a center word and thus capture the idea that any given word should be predictive of nearby words.

* The nCBOW models predict the center word from nearby context words and thus capture the idea that a word should be predictable based on its context.

*   The kNS models are a trick to save memory over the FS models. Rather than computing a probability for every single word in the vocabulary every time we want to predict a target word (center word for CBOW and context word for SG, respectively), we just predict whether the target word is likely and whether k randomly sampled words are likely. This mimics the behavior of a full softmax prediction, which trains the model to up the probability of the target word and lower the probability of all other words. In the case of kNS, we up the probability of the target word and lower the probability of k other words.




# Quiz Question 8:
What is the last index in the SST example after numericalization?

In [0]:
sentence_idx = random.randint(0, len(en_wiki_numericalized)-1)   
#print(sentence_idx)
#print(len(en_wiki_numericalized))
sentence = en_wiki_numericalized[sentence_idx]
#print(len(sentence))
window_idx = random.randint(0, len(sentence)-1)
window = sentence[window_idx:(window_idx+5)]
print(sentence[window_idx])
#print(window_idx)
print(window)
#for i in range(4):
#  print(en_wiki_vocab.index_to_word[window[i]])
context_words = list(window)
center_word = list([context_words.pop(len(window)//2)])
#center_word = window[len(window)//2]
print(len(window))
print(id(window))
print(id(context_words))
#context_words.remove(center_word)
print(len(window))
print(center_word)
print([context_words,center_word])
ex = context_words
ex.append(center_word)
print(type(ex))
print(context_words)
print(type(center_word))
context_word = [context_words[random.randint(0, len(context_words)-1)]]
print([center_word, context_word])
print(type(context_word))
#example = []
ss = [random.randint(0, len(en_wiki_vocab.index_to_word)-1) for _ in range(15)]
print(type(ss))

print(ex + ss)

2110
[2110, 8463, 64, 853, 16]
5
139874246640776
139874251192712
5
[64]
[[2110, 8463, 853, 16], [64]]
<class 'list'>
[2110, 8463, 853, 16, [64]]
<class 'list'>
[[64], [853]]
<class 'list'>
<class 'list'>
[2110, 8463, 853, 16, [64], 4523, 65283, 64773, 51286, 33523, 61373, 58832, 33559, 34068, 71045, 2270, 24031, 57249, 36386, 13525]


# Hands-on: Construct examples for each W2V variant

In [0]:

def construct_examples(numericalized_sentences, vocabulary, num_examples=int(1e6), n=5, sg=True, k=0):
  examples = []
  while True:
    # TODO: select a random sentence index using random.randint and get that
    # sentence. Be careful to avoid indexing errors.
    sentence_idx = random.randint(0, len(numericalized_sentences)-1)   
    sentence = numericalized_sentences[sentence_idx]
    # TODO: Select a random window index using random.randint
    # and obtain that window of size n. Be careful to avoid indexing errors.
    window_idx = random.randint(0, len(sentence)-1)
    window = sentence[window_idx:window_idx+n]

    
    if len(window) <= n//2:
      continue
      
    # TODO: Get the center word and the context words 
    context_words = list(window)
    center_word = context_words.pop(len(window)//2)
    if sg: # if 
      context_word = context_words[random.randint(0, len(context_words)-1)]
      example = [[center_word], context_word]
      #example.append(context_word)
    else: # if CBOW
      example = [context_words]
      example.append(center_word)
      if len(window) < n:
        continue
      
    if k > 0: # if doing negative sampling
      samples = [random.randint(0, len(vocabulary.index_to_word)-1) 
                 for _ in range(k)]
      example.append(samples)
      
    examples.append(example)
    if len(examples) >= num_examples:
      break
  
  return examples

In this exercise, you'll implement parts of the ```construct_examples``` method. First, grab a random sentence from your numericalized vocabulary and construct your context window of size n from a random word in that sentence. Next, pick your center and context words. Finally, create examples for the skip-gram and CBOW models.

In [0]:

random.seed(123)

print('Constructing English Wikipedia examples for 5SG-FS model')
en_wiki_5sgfs_examples = construct_examples(en_wiki_numericalized, en_wiki_vocab)
print('Constructing English Wikipedia examples for 5CBOW-FS model')
en_wiki_5cbowfs_examples = construct_examples(en_wiki_numericalized, en_wiki_vocab, sg=False)
print('Constructing English Wikipedia examples for 5SG-15NS model')
en_wiki_5sg15ns_examples = construct_examples(en_wiki_numericalized, en_wiki_vocab, k=15)
print('Constructing English Wikipedia examples for 5CBOW-15NS model')
en_wiki_5cbow15ns_examples = construct_examples(en_wiki_numericalized, en_wiki_vocab, sg=False, k=15)

Constructing English Wikipedia examples for 5SG-FS model
Constructing English Wikipedia examples for 5CBOW-FS model
Constructing English Wikipedia examples for 5SG-15NS model
Constructing English Wikipedia examples for 5CBOW-15NS model


In [0]:
# random.seed(123)
print('Constructing Stanford Sentiment Treebank examples for 5SG-15NS model')
sst_5sgfs_examples = construct_examples(sst_numericalized, sst_vocab)
print('Constructing Stanford Sentiment Treebank examples for 5CBOW-15NS model')
sst_5cbowfs_examples = construct_examples(sst_numericalized, sst_vocab, sg=False)
print('Constructing Stanford Sentiment Treebank examples for 5SG-15NS model')
sst_5sg15ns_examples = construct_examples(sst_numericalized, sst_vocab, k=15)
print('Constructing Stanford Sentiment Treebank examples for 5CBOW-15NS model')
sst_5cbow15ns_examples = construct_examples(sst_numericalized, sst_vocab, sg=False, k=15)

Constructing Stanford Sentiment Treebank examples for 5SG-15NS model
Constructing Stanford Sentiment Treebank examples for 5CBOW-15NS model
Constructing Stanford Sentiment Treebank examples for 5SG-15NS model
Constructing Stanford Sentiment Treebank examples for 5CBOW-15NS model


**USE  the next cell only when  num_examples​ is small. Otherwise it will crash**

In [0]:
print(en_wiki_5sgfs_examples[1])
print(en_wiki_5cbowfs_examples[1])
print(en_wiki_5sg15ns_examples[1])
print(en_wiki_5cbow15ns_examples[1])
print(sst_5sgfs_examples[1])
print(sst_5cbowfs_examples[2])
print(sst_5sg15ns_examples[1])
print(sst_5cbow15ns_examples[1])

[[1918], 1918]
[[2743, 889, 7828, 183], 815]
[[12570], 5214, [18669, 31842, 49242, 2615, 4400, 3751, 35429, 23222, 26546, 57992, 49827, 59321, 54399, 17022, 13815]]
[[855, 3661, 9276, 613], 643, [33221, 3192, 47469, 58323, 8941, 32774, 63262, 8683, 11388, 24960, 44014, 56184, 29325, 8084, 42301]]
[[735], 120]
[[2440, 601, 592, 196], 2752]
[[0], 955, [1962, 1044, 930, 1828, 347, 367, 839, 1091, 2279, 1010, 3217, 1414, 726, 1896, 2356]]
[[830, 2433, 19, 2530], 1439, [1195, 2027, 1607, 2206, 1656, 1489, 2056, 2574, 1710, 1116, 1374, 1843, 2950, 1448, 611]]


# Construct minibatches for the FS models
A minibatch consists of multiple examples grouped together as torch tensors. We want to batch the inputs and the targets so the model can process them in parallel.

If we're using a full softmax, minibatching is very simple. The inputs are whatever the first entry of an example is and the targets are whatever the second entry in the example is. This should be as expected.

In [0]:
def tensorize(x, dtype=torch.long):
  if len(x) == 0:
    return None
  return torch.tensor(x, dtype=dtype)

def batchFS(examples, batch_size=64):
  example_indices = random.sample(range(0, len(examples)-1), batch_size)
  batch_examples = [examples[idx] for idx in example_indices]
  inputs = [example[0] for example in batch_examples]
  targets = [example[1] for example in batch_examples]
  return [tensorize(inputs), tensorize(targets)]


In [0]:
random.seed(123)
for x in batchFS(sst_5sgfs_examples):
  print(x)

tensor([[ 1331],
        [  504],
        [  461],
        [ 1620],
        [ 1955],
        [ 2450],
        [  177],
        [ 2713],
        [ 2730],
        [  484],
        [ 1089],
        [ 2277],
        [  248],
        [  570],
        [  143],
        [  637],
        [ 2253],
        [ 1118],
        [ 3242],
        [   11],
        [  854],
        [ 1931],
        [ 1156],
        [ 1987],
        [  261],
        [  820],
        [  253],
        [    1],
        [ 3022],
        [ 2453],
        [   38],
        [ 2111],
        [  658],
        [  527],
        [  439],
        [ 1710],
        [ 2245],
        [ 3134],
        [  541],
        [ 1802],
        [  902],
        [  382],
        [   23],
        [  535],
        [  184],
        [ 1294],
        [  389],
        [  583],
        [  606],
        [  208],
        [ 2327],
        [  776],
        [ 1190],
        [ 1320],
        [ 2694],
        [  963],
        [  920],
        [  804],
        [  812

# Quiz Question 9:
What is the last index in the last tensor printed out above for the batched examples?

# Construct minibatches for the kNS models
Negative sampling requires three tensors instead of two. The first tensor represents the inputs, just like before. The second is an output tensor with each row containing first the target word index and then the negative sample word indices. The third is a labels tensor representing each of the entries in the output tensor as a 1 if it is the target word and a 0 if it is a negative sample. 

In [0]:
#batch_examples = [[[2765], [1273,234,87,78], [2435, 2531, 1750, 1925, 1739, 958, 825, 2676, 1651, 2091, 2901, 2573, 2307, 350, 1034],[[2765], [1273,234,87,78], [2435, 2531, 1750, 1925, 1739, 958, 825, 2676, 1651, 2091, 2901, 2573, 2307, 350, 1034]]
batch_examples = [[1,2,3],[4,5,6,7]]
#example_indices = random.samplerange(0, len(en_wiki_5cbow15ns_examples)-1), 2)
#batch_examples = [en_wiki_5cbow15ns_examples[idx] for idx in example_indices]
inputs = [example[0] for example in batch_examples]
targets = [example[1] for example in batch_examples]
negatives = [example[2] for example in batch_examples]
dx = tensorize(targets).unsqueeze(1)
dy = tensorize(negatives)
outputs = torch.cat([tensorize(targets).unsqueeze(1), tensorize(negatives).unsqueeze(1)], dim=1)
labels = torch.zeros_like(outputs, dtype=torch.float)
labels[:, 0] = 1.
print(batch_examples)
print(inputs)
print(targets)
print(negatives)
print(labels)
print(outputs)
#print(dy)
x = torch.Tensor([4,6, 2, 3]).unsqueeze(1)
r = torch.Tensor([1, 2]).unsqueeze(1)
print(x)
print(r)
#y = torch.cat((x,r), dim=1)

print(y)

[[1, 2, 3], [4, 5, 6, 7]]
[1, 4]
[2, 5]
[3, 6]
tensor([[ 1.,  0.],
        [ 1.,  0.]])
tensor([[ 2,  3],
        [ 5,  6]])
tensor([[ 4.],
        [ 6.],
        [ 2.],
        [ 3.]])
tensor([[ 1.],
        [ 2.]])


NameError: ignored

In [0]:
def batchkNS(examples, batch_size=64):
  example_indices = random.sample(range(0, len(examples)-1), batch_size)
  batch_examples = [examples[idx] for idx in example_indices]
  inputs = [example[0] for example in batch_examples]
  targets = [example[1] for example in batch_examples]
  negatives = [example[2] for example in batch_examples]
  outputs = torch.cat([tensorize(targets).unsqueeze(1), tensorize(negatives)], dim=1)
  labels = torch.zeros_like(outputs, dtype=torch.float)
  labels[:, 0] = 1.
  
  return [tensorize(inputs), outputs, labels]

In [0]:
random.seed(123)
for x in batchkNS(en_wiki_5cbow15ns_examples):
  print(x)

tensor([[  2293,   1280,   7338,    309],
        [   223,   2699,    809,    982],
        [ 22049,   1332,  22204,  58120],
        [ 10956,   8949,   2782,    432],
        [   766,  28414,   1868,   8894],
        [     7,   2095,  35820,   1801],
        [  6915,    116,    178,   8884],
        [  4996,     10,  16329,    668],
        [   526,  35797,  22189,  24886],
        [    11,      5,   3430,  11658],
        [   419,      7,    419,   1855],
        [  1593,    584,   1998,   1890],
        [   157,  49383,    195,    250],
        [ 11529,   4137,   1021,   4408],
        [  6144,   8321,   2906,    421],
        [    20,     56,  14695,  41501],
        [  3360,   3388,  20896,   1093],
        [  1672,   2248,    760,    285],
        [   438,   9526,  38504,  31897],
        [  6229,  47492,   3947,   4643],
        [    31,   2390,   5351,   1916],
        [ 37656,   4758,  37656,    700],
        [     2,    517,    563,   5022],
        [  5546,   1491,  41625,  

# Quiz Question 10:
What is the first index in the first tensor printed out above for the batched examples?

# Construct a generic batch function
We can now wrap our two batch construction functions in a way that's generic and adapts to the kind of examples we pass in. Later on, this flexibility will be very convenient. You can verify that as long as the random seed is the same, the batches are in fact identical to those constructed earlier.

In [0]:
def batch(examples, batch_size=64):
  assert len(examples) > 0
  
  if len(examples[0]) == 2:
    return batchFS(examples, batch_size=batch_size)
  else:
    return batchkNS(examples, batch_size=batch_size)
 
random.seed(123)
for x in batch(sst_5sgfs_examples):
  print(x)
print()
random.seed(123) 
for x in batch(en_wiki_5cbow15ns_examples):
  print(x)

tensor([[ 1331],
        [  504],
        [  461],
        [ 1620],
        [ 1955],
        [ 2450],
        [  177],
        [ 2713],
        [ 2730],
        [  484],
        [ 1089],
        [ 2277],
        [  248],
        [  570],
        [  143],
        [  637],
        [ 2253],
        [ 1118],
        [ 3242],
        [   11],
        [  854],
        [ 1931],
        [ 1156],
        [ 1987],
        [  261],
        [  820],
        [  253],
        [    1],
        [ 3022],
        [ 2453],
        [   38],
        [ 2111],
        [  658],
        [  527],
        [  439],
        [ 1710],
        [ 2245],
        [ 3134],
        [  541],
        [ 1802],
        [  902],
        [  382],
        [   23],
        [  535],
        [  184],
        [ 1294],
        [  389],
        [  583],
        [  606],
        [  208],
        [ 2327],
        [  776],
        [ 1190],
        [ 1320],
        [ 2694],
        [  963],
        [  920],
        [  804],
        [  812

# Hands-on: Define the Word2Vec models
We'll define a single model that handles all variants of Word2Vec because there is so much overlap in how the models work. A Word2VecModel has a set of vectors that are trainable parameters. These are of some size embedding_size. The first half of these vectors is treated as input embeddings for each word, and the second half is treated as output embeddings (either for the softmax or the partial softmax used in negative sampling). These vectors are randomly initialized, but trained using one of the Word2Vec variants. Ultimately, the models return a tensor of scores with size of either the entire vocabulary (in the case of FS, scores will have size batch_size x vocabulary_size) or over the target words and their respective negative samples (in this case, scores will have size batch_size x (k +1)).

The inline comments guide you through this process in detail.

In [0]:
vec = nn.Parameter(torch.randn(30))
print(vec)

Parameter containing:
tensor([-1.5205, -1.1683,  1.3898, -0.0192, -0.7163, -1.3071,  0.0507,
        -0.4991, -0.3405, -1.3760,  1.4353, -0.8220,  0.9990, -1.1907,
        -0.5474,  0.9044,  0.2522, -1.1754,  0.0408, -0.1454,  0.2937,
         0.2052, -0.6712, -0.4653, -1.4452, -0.7696, -0.4504, -1.5837,
         1.2553,  1.0233])


In [0]:
class Word2VecModel(nn.Module):
  
  def __init__(self, vocab_size, embedding_size=300):
    super().__init__()
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    # TODO: Use randn to initialize a tensor for vocab_size vectors each
    # of size embedding_size
    vectors = torch.randn(embedding_size)
    self.vectors = nn.Parameter(vectors)
    
  def forward(self, batch, samples=None):
    inputs = batch[0]
    # TODO: Obtain the input vector portion (the first self.embedding_size//2
    # entries of the input vectors) of self.vectors for inputs
    input_vectors = 
    
    # TODO: If this is a CBOW model, 
    # compute a continous bag-of-words over the input vectors
    if input_vectors.dim() == 3:
      input_vectors = 

    if len(batch) == 2: # Full Softmax
      # TODO: obtain the output portion of all vectors 
      # HINT: you can index into the last dimension of tensors with any number 
      # of dimensions by using the following notation tensor[..., idx]
      target_vectors = 
      # TODO: compute scores between input and output vectors
      # via matrix multiplication (you'll need to transpose output_vectors)
      scores = 
    else: # Negative Sampling
      outputs = batch[1]
      # TODO: obtain the output vectors only for samples
      output_vectors = 
      # TODO: compute scores between the input vectors and the output vectors
      # First, you'll need to expand the input vectors along dimension 1 using 
      # unsqueeze() so that the input vectors are now of shape (batch_size, 1, embedding_size)
      input_vectors = 
      # Then you'll need to transpose the output_vectors along dimensions 1 and 2
      # so that the output_vectors is of shape (batch_size, embedding_size, k + 1)
      output_vectors = 
      # Now a matrix multiply should yield a tensor of size (batch_size, 1, k + 1)
      # and you can get the matrix of scores by calling squeeze() to get a tensor
      # of size (batch_size, k + 1), which should match the size of labels
      scores = 
    return scores

# Training Word2Vec
Now we need to define a training loop that trains our models for some number of iterations. The inline comments walk you through this process.

In [0]:
def get_trainable_parameters(model):
  """Returns the trainable parameters of a model"""
  return list(filter(lambda p: p.requires_grad, model.parameters()))

def denumericalize(vocab, x):
  return [vocab.index_to_word[y] for y in x]


def train(model, vocab, dataset, device, max_iterations=int(1e6), log_every=1e4,
          val_every=1e5, num_val_samples=50, val_head=500, num_neighbors=5,
          anneal_every=int(1e8), batch_size=256):
  """Trains a Word2VecModel on a Word2VecDataset for max_iterations (int)"""
  print('Training on ', len(dataset), ' examples for '
        , max_iterations, ' iterations with ', len(vocab.index_to_word), ' words in the vocabulary.')
  model.to(device)
  model.train()
  # TODO: initialize a default Adam optimizer
  # hint: use get_trainable_parameters
  opt =  
  avg_loss = 0
  for iteration in range(max_iterations):
    # TODO: zero out the gradients the optimizer is tracking
    
    # TODO: get the next batch from the dataset
    b = 
    b = [x.to(device) for x in b]
    # TODO: get scores from the model
    scores = 

    if len(b) == 2: # Full Softmax
      targets = b[1]
      # TODO: use scores and targets to compute a cross entropy loss
      loss = 
    else:
      labels = b[2]
      # TODO: use the scores and labels to compute the loss using
      # a binary cross entropy with logits loss function
      loss = 
    
    # TODO: compute gradients using the loss

    # TODO: update your parameters by using the optimizer to take a step
    
    
    # Annealing the learning rate
    if (iteration + 1) % anneal_every == 0:
      print('Annealing')
      opt.param_groups[0]['lr'] *= 0.5
      
    # logging  
    avg_loss += loss.item()
    if (iteration + 1) % log_every == 0:
      print(f'Iteration: {iteration + 1}, avg_loss: {avg_loss / log_every}')
      avg_loss = 0
      
    # validating by checking nearest neighbors
    if (iteration + 1) % val_every == 0:
      model.eval()
      print('\nValidating:\n')
      normalized_vectors = F.normalize(model.vectors)
      random_samples = random.sample(range(int(val_head)), num_val_samples)
      valid_scores = torch.matmul(normalized_vectors[random_samples], normalized_vectors.transpose(0, 1))
      nearest_neighbors = valid_scores.detach().cpu().numpy().argsort(axis=1)[:, -(num_neighbors+1):-1]
      for i, row in enumerate(nearest_neighbors):
        key_word = denumericalize(vocab, [random_samples[i]])
        neighbors = denumericalize(vocab, row.tolist())
        neighbors.reverse()
        print(f'word: {key_word} --', f'neighbors: {neighbors}') 
      print()

# Train Skip-Gram models

Below are some examples of how to use the code we've written to train word vectors. Often it takes much more data and a large number of iterations in order to get high quality word vectors, but you can see some reasonable patterns developing using the examples below. Notice that the word vectors from English Wikipedia might give more intuitive neighbors than those from SST because Wikipedia is a much larger dataset and less domain specific.

## Note: Enable GPU
Training models takes a long time. You can speed it up by going to **Edit > Notebook Settings** and changing the hardware accelerator to **GPU**. Leave this setting for the remainder of the exercise to speed up the code runs.

# Full softmax

Run the code below to train word vectors with full softmax for our SST dataset.

#### Stanford Sentiment Treebank

In [0]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
random.seed(123)
device = torch.device("cuda:0")

vocab_size = len(sst_vocab.index_to_word)
start_time = time.time()
train(Word2VecModel(vocab_size=vocab_size), sst_vocab, sst_5sgfs_examples, device)
print('Time Elapsed: ', time.time() - start_time)

# Quiz Question 11:
What is the the fifth nearest neighbor (the one all the way to the right) for 'suspense'?

# Quiz Question 12:
What is the the  nearest neighbor (the one all the way to the left) for 'death'?

#### English Wikipedia

In [0]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
random.seed(123)
device = torch.device("cuda:0")

vocab_size = len(en_wiki_vocab.index_to_word)
start_time = time.time()
train(Word2VecModel(vocab_size=vocab_size), en_wiki_vocab, en_wiki_5sgfs_examples, device, log_every=1000, val_every=10000, max_iterations=100000)
print('Time Elapsed: ', time.time() - start_time)

# Quiz Question 13:
What are the nearest neighbors for 'minister'?

# Train CBOW models

There won't be quiz questions on these models as they're a bit less reliable than the full softmax, skip-gram models, but we encourage you to play around with them!

### Full Softmax

#### Stanford Sentiment Treebank

In [0]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
random.seed(123)
device = torch.device("cuda:0")

vocab_size = len(sst_vocab.index_to_word)
start_time = time.time()
train(Word2VecModel(vocab_size=vocab_size), sst_vocab, sst_5cbowfs_examples, device)
print('Time Elapsed: ', time.time() - start_time)

### Negative Sampling

#### English Wikipedia

In [0]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
random.seed(123)
device = torch.device("cuda:0")

vocab_size = len(en_wiki_vocab.index_to_word)
start_time = time.time()
train(Word2VecModel(vocab_size=vocab_size), en_wiki_vocab, en_wiki_5cbow15ns_examples, device, log_every=1000,  val_every=10000, max_iterations=100000)
print('Time Elapsed: ', time.time() - start_time)